# CRIME IN LOS ANGELES
    Crime data from 2010 through September 2017

In [1]:
#TEAM MEMBERS:
#              Sylvester Zowonu
#              Frank Navarette
#              Max Ru
#Course      : CS 4661 (Data Science)
#Instructor  : Dr. Mohammad (Mo) Pourhomayoun 
#
#  
# 
# File Locations : Crime Data https://www.dropbox.com/s/rhnsu6p95clsojk/Crime_Data_2010_2017.csv?dl=0
#                  MO_Codes   https://www.dropbox.com/s/e3wuydmcgwto5ox/MO_Codes.csv?dl=0
# Instructions: Files should be in the same directory of this file

Import Packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import datetime as dt
from pylab import rcParams
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
filename = 'Crime_Data_2010_2017.csv'
crime_data = pd.DataFrame()
crime_data = pd.read_csv(filename,nrows=100000)

In [3]:
crime_data.head()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
0,1208575,03/14/2013,03/11/2013,1800,12,77th Street,1241,626,INTIMATE PARTNER - SIMPLE ASSAULT,0416 0446 1243 2000,...,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AO,Adult Other,626.0,NaN,NaN,NaN,6300 BRYNHURST AV,NaN,"(33.9829, -118.3338)"
1,102005556,01/25/2010,01/22/2010,2300,20,Olympic,2071,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,VAN NESS,15TH,"(34.0454, -118.3157)"
2,418,03/19/2013,03/18/2013,2030,18,Southeast,1823,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,200 E 104TH ST,NaN,"(33.942, -118.2717)"
3,101822289,11/11/2010,11/10/2010,1800,18,Southeast,1803,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,88TH,WALL,"(33.9572, -118.2717)"
4,42104479,01/11/2014,01/04/2014,2300,21,Topanga,2133,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329,...,NaN,IC,Invest Cont,745.0,NaN,NaN,NaN,7200 CIRRUS WY,NaN,"(34.2009, -118.6369)"


In [4]:
crime_data.shape

(100000, 26)

# Crime types

In [5]:
crime_data['Crime Code Description'].value_counts().head(20)

BATTERY - SIMPLE ASSAULT                                        10154
THEFT PLAIN - PETTY ($950 & UNDER)                               9114
BURGLARY FROM VEHICLE                                            8287
VEHICLE - STOLEN                                                 7929
BURGLARY                                                         7086
THEFT OF IDENTITY                                                5769
INTIMATE PARTNER - SIMPLE ASSAULT                                5278
VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS) 0114     4801
VANDALISM - MISDEAMEANOR ($399 OR UNDER)                         4694
THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)                  3958
THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD0036     3913
ROBBERY                                                          3579
ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT                   3526
CRIMINAL THREATS - NO WEAPON DISPLAYED                           2952
SHOPLIFTING - PETTY 

# Count of Victim Gender

In [6]:
crime_data['Victim Sex'].value_counts()

M    48128
F    41618
X     1017
H        7
Name: Victim Sex, dtype: int64

# Count by Vicitim Race

In [7]:
Victims_bg = {
    "A": "Asian",
    "B": "Black",
    "C": "Chinese",
    "D": "Cambodian",
    "F": "Filipino",
    "G": "Guamanian",
    "H": "Hispanic/Latin/Mexican",
    "I": "American Indian/Alaskan Native",
    "J": "Japanese",
    "K": "Korean",
    "L": "Laotian",
    "O": "Other",
    "P": "Pacific Islander",
    "S": "Samoan",
    "U": "Hawaiian",
    "V": "Vietnamese",
    "W": "White",
    "X": "Unknown",
    "Z": "Asian Indian"
}
#crime_data["Victim Descent"] = crime_data["Victim Descent"].map(Victims_bg)
crime_data['Victim Descent'].value_counts()

H    32024
W    28866
B    14270
O    10898
A     2459
X     1785
F      149
K      149
C       63
I       56
J       18
P       12
U        7
Z        5
G        3
V        3
S        1
L        1
Name: Victim Descent, dtype: int64

# Type of Weapons used

In [8]:
crime_data['Weapon Description'].value_counts().head(20)

STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)    19649
VERBAL THREAT                                      2862
UNKNOWN WEAPON/OTHER WEAPON                        2105
HAND GUN                                           1309
KNIFE WITH BLADE 6INCHES OR LESS                    617
SEMI-AUTOMATIC PISTOL                               501
OTHER KNIFE                                         410
VEHICLE                                             283
UNKNOWN FIREARM                                     277
BOTTLE                                              267
REVOLVER                                            247
ROCK/THROWN OBJECT                                  177
STICK                                               177
CLUB/BAT                                            175
BLUNT INSTRUMENT                                    173
SIMULATED GUN                                       172
KNIFE WITH BLADE OVER 6 INCHES IN LENGTH            168
FOLDING KNIFE                                   

# Crime by periods [Day, Month, Year]

In [9]:
#convert date reported to date object
try:
    date_occ = [dt.datetime.strptime(d, "%m/%d/%Y").date() for d in crime_data["Date Occurred"]]
except:
    print("Error Date Format")
crime_data["Date Occurred"] = np.array(date_occ)

day = [data.isoweekday() for data in crime_data["Date Occurred"]]
mon = [data.month for data in crime_data["Date Occurred"]]
year = [data.year for data in crime_data["Date Occurred"]]

crime_data["Day Occurred"] = np.array(day)
crime_data["Month Occurred"] = np.array(mon)
crime_data["Year Occurred"] = np.array(year)

In [10]:
#Day of The Week
week_day = { 
    1:"Monday",
    2:"Tuesday",
    3:"Wednesday",
    4:"Thursday",
    5:"Friday",
    6:"Saturday",
    7:"Sunday"
}
#crime_data['Day Occurred'] = crime_data['Day Occurred'].map(week_day)
crime_data['Day Occurred'].value_counts()

5    15724
4    14229
3    14186
1    14153
6    14150
2    14130
7    13428
Name: Day Occurred, dtype: int64

In [11]:
#Month of The Year
month_year = { 
    1:"January",
    2:"February",
    3:"March",
    4:"April",
    5:"May",
    6:"June",
    7:"July",
    8:"August",
    9:"September",
    10:"October",
    11:"November",
    12:"December"
}
#crime_data['Month Occurred'] = crime_data['Month Occurred'].map(month_year)
crime_data['Month Occurred'].value_counts()

1     9209
8     9015
7     8679
5     8669
3     8664
6     8390
9     8367
4     8262
10    8246
11    7823
2     7573
12    7103
Name: Month Occurred, dtype: int64

In [12]:
crime_data['Month Occurred'].describe()[1:]

mean     6.390050
std      3.404621
min      1.000000
25%      3.000000
50%      6.000000
75%      9.000000
max     12.000000
Name: Month Occurred, dtype: float64

In [13]:
#Crime by the Years
crime_data['Year Occurred'].value_counts()

2013    83188
2010    16669
2014      143
Name: Year Occurred, dtype: int64

In [14]:
crime_data['Victim Age'].describe()[1:]

mean    36.402443
std     16.807675
min     10.000000
25%     24.000000
50%     34.000000
75%     48.000000
max     99.000000
Name: Victim Age, dtype: float64

In [15]:
crime_data['Premise Description'].value_counts().head(20)

STREET                                          22126
SINGLE FAMILY DWELLING                          18738
MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)    14120
PARKING LOT                                      7121
SIDEWALK                                         5537
OTHER BUSINESS                                   4997
VEHICLE, PASSENGER/TRUCK                         3337
GARAGE/CARPORT                                   2110
DRIVEWAY                                         1725
RESTAURANT/FAST FOOD                             1358
DEPARTMENT STORE                                 1327
MARKET                                           1059
OTHER STORE                                       920
PARK/PLAYGROUND                                   910
HIGH SCHOOL                                       867
PARKING UNDERGROUND/BUILDING                      700
ALLEY                                             668
DRUG STORE                                        652
HOTEL                       

In [16]:
crime_data['Status Description'].value_counts().head(20)

Invest Cont     74616
Adult Other     12665
Adult Arrest    11538
Juv Arrest        928
Juv Other         252
UNK                 1
Name: Status Description, dtype: int64

In [17]:
crime_data['Status Code'].value_counts().head(20)

IC    74616
AO    12665
AA    11538
JA      928
JO      252
Name: Status Code, dtype: int64

OneHot Encoding function

In [18]:
def one_hot(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode 
    @return a DataFrame with one-hot encoding
    """
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    return df

In [19]:
crime_data['Area Name'].value_counts().head(20)
#list(crime_data)

West Valley    14592
Southwest      12223
Van Nuys        8998
Hollywood       8247
Harbor          8205
Northeast       8059
Rampart         7889
West LA         7774
Wilshire        7510
Central         7373
Hollenbeck      6462
77th Street     1311
Newton           328
Pacific          289
Mission          146
Olympic          125
Devonshire       104
Foothill         102
Southeast        101
Topanga           88
Name: Area Name, dtype: int64

In [20]:
#crime_data['Area ID'].value_counts().head(20)

In [21]:
#crime_data['Reporting District'].value_counts().head(20)

In [22]:
crime_data.dtypes

DR Number                   int64
Date Reported              object
Date Occurred              object
Time Occurred               int64
Area ID                     int64
Area Name                  object
Reporting District          int64
Crime Code                  int64
Crime Code Description     object
MO Codes                   object
Victim Age                float64
Victim Sex                 object
Victim Descent             object
Premise Code              float64
Premise Description        object
Weapon Used Code          float64
Weapon Description         object
Status Code                object
Status Description         object
Crime Code 1              float64
Crime Code 2              float64
Crime Code 3              float64
Crime Code 4              float64
Address                    object
Cross Street               object
Location                   object
Day Occurred                int64
Month Occurred              int64
Year Occurred               int64
dtype: object

In [23]:
feature_cols = ['Area ID',
                'Day Occurred',
                'Month Occurred',
                'Year Occurred',
                'Time Occurred',
                'Reporting District',
                'Victim Age',
                'Victim Sex',
                'Victim Descent',
                'Premise Code',
                'Weapon Used Code',
                'Status Code']
X = crime_data[feature_cols]
X = one_hot(X,feature_cols)
X = X.drop(feature_cols,axis=1)
X.shape

(100000, 2567)

In [24]:
y = crime_data['Crime Code']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

# KNN Model

In [26]:
knn = KNeighborsClassifier(n_neighbors=21) 

In [31]:
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=21, p=2,
           weights='uniform')

In [32]:
predict_y = knn.predict(X_test)
score = accuracy_score(y_test, predict_y)

# Accuracy

In [33]:
print('Accracy score {0:.2f}'.format(score*100))

Accracy score 38.41


# Random Forest Model

In [27]:
rf_clf = RandomForestClassifier(n_jobs=2, random_state=0,verbose=1)
rf_clf.fit(X_train,y_train)

[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    9.1s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=2, oob_score=False, random_state=0,
            verbose=1, warm_start=False)

In [28]:
y_predict_rf = rf_clf.predict(X_test)
score_rf = accuracy_score(y_test, y_predict_rf)

[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.3s finished


# Accuracy

In [29]:
print('Accracy score {0:.2f}'.format(score_rf*100))

Accracy score 44.35


In [30]:
rf_clf.feature_importances_

array([ 0.0038344 ,  0.00331344,  0.00487403, ...,  0.01131354,
        0.00105924,  0.00037424])